In [1]:
import mysql.connector
from functools import partial


def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='localhost',
        user='megadados',
        password='megadados',
        database='agro',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

In [2]:
#Fazendo um teste:

db('''
SELECT
    *
FROM 
    fazenda
''')

db('''
SELECT
    *
FROM 
    plantacao
''')

db('''
SELECT
    *
FROM 
    inseticida
''')

db('''
SELECT
    *
FROM 
    lote
''')

db('''
SELECT
    *
FROM 
    plantacao_inseticida
''')

Executando query:
(1, 'Terra Forte')
(2, 'Agua Brava')
Executando query:
(1, 'milho')
(2, 'soja')
(3, 'cafe')
(4, 'algodao')
Executando query:
(1, 'Mata-Tudo', Decimal('200.00'))
(2, 'Mae-Terra', Decimal('500.00'))
(3, 'Carinho', Decimal('700.00'))
Executando query:
(1, 1, 2, 3, 100.0)
(2, 1, 1, 2, 70.0)
(3, 1, None, None, 80.0)
(4, 2, 4, None, 90.0)
(5, 2, 3, 3, 50.0)
(6, 2, 2, 2, 30.0)
(7, 2, 4, None, 110.0)
Executando query:
(1, 1, 2)
(1, 2, 1)
(2, 2, 1)
(2, 3, 1)
(3, 1, 4)
(3, 3, 1)


### QUESTÃO 4)

a) Crie 3 lotes para a fazenda: um de 4 hectares, outro de 3 hectares, e outro
de 5 hectares.

In [3]:
db('''
INSERT INTO Lote (id_lote, id_fazenda, id_plantacao, id_inseticida, area)
VALUES (8,2,1,2,4) , (9,1,2,3,3) , (10,2,3,1,5)''')

Executando query:


In [4]:
db('''
SELECT
    id_lote, id_fazenda, id_plantacao, id_inseticida, area
FROM 
    Lote
''')

Executando query:
(1, 1, 2, 3, 100.0)
(2, 1, 1, 2, 70.0)
(3, 1, None, None, 80.0)
(4, 2, 4, None, 90.0)
(5, 2, 3, 3, 50.0)
(6, 2, 2, 2, 30.0)
(7, 2, 4, None, 110.0)
(8, 2, 1, 2, 4.0)
(9, 1, 2, 3, 3.0)
(10, 2, 3, 1, 5.0)


b) Corrija um erro no item anterior: mude o tamanho do lote de 3 hectares para 3,5 hectares.


In [5]:
db(''' UPDATE Lote SET area = "3.5" WHERE id_lote = 9 ''')

Executando query:


In [6]:
db('''
SELECT
    id_lote, id_fazenda, id_plantacao, id_inseticida, area
FROM 
    Lote
''')

Executando query:
(1, 1, 2, 3, 100.0)
(2, 1, 1, 2, 70.0)
(3, 1, None, None, 80.0)
(4, 2, 4, None, 90.0)
(5, 2, 3, 3, 50.0)
(6, 2, 2, 2, 30.0)
(7, 2, 4, None, 110.0)
(8, 2, 1, 2, 4.0)
(9, 1, 2, 3, 3.5)
(10, 2, 3, 1, 5.0)


c) Como obter o id do lote em uma aplicação real? É realista pedir ao usuário que decore o id de cada lote?


**R**: Não, não é realista pedir para o usuário decorar o id de cada lote. O que pode ser feito é criar tabelas e utilizar ferramentas como o SELECT para obter as informações desejadas. Ou seja, a aplicação deve ter uma interface de usuário que permita a seleção do lote de interesse de modo simples.


### Questão 5

Escreva queries para as seguintes tarefas:
    
a) Qual a maior fazenda?

In [7]:
# DEBUGGGG

db('''
SELECT 
    id_fazenda, nome, id_lote, area
FROM
    fazenda
    INNER JOIN lote USING (id_fazenda)    
''')

Executando query:
(1, 'Terra Forte', 1, 100.0)
(1, 'Terra Forte', 2, 70.0)
(1, 'Terra Forte', 3, 80.0)
(1, 'Terra Forte', 9, 3.5)
(2, 'Agua Brava', 4, 90.0)
(2, 'Agua Brava', 5, 50.0)
(2, 'Agua Brava', 6, 30.0)
(2, 'Agua Brava', 7, 110.0)
(2, 'Agua Brava', 8, 4.0)
(2, 'Agua Brava', 10, 5.0)


In [8]:
# RESPOSTA MINHAAAA:

db('''
SELECT 
    id_fazenda, nome, SUM(area) as area_total
FROM
    fazenda
    INNER JOIN lote USING (id_fazenda)
GROUP BY
    id_fazenda
ORDER BY
    area_total DESC
LIMIT
    1
''')

Executando query:
(2, 'Agua Brava', 289.0)


In [9]:
# RESPOSTA DO PROF (Não funciona quando tenho mais que uma fazendaaaa)

db('''
SELECT
    nome, SUM(area) AS total
FROM
    fazenda INNER JOIN lote USING (id_fazenda)
ORDER BY 
    total DESC
LIMIT 1;
''')


Executando query:
('Terra Forte', 542.5)


b) Quais as plantações com área total maior que 10 hectares?

In [10]:
# Antes, só conferir os lotes que tão plantados:

db('''
SELECT
    *
FROM 
    lote
''')


Executando query:
(1, 1, 2, 3, 100.0)
(2, 1, 1, 2, 70.0)
(3, 1, None, None, 80.0)
(4, 2, 4, None, 90.0)
(5, 2, 3, 3, 50.0)
(6, 2, 2, 2, 30.0)
(7, 2, 4, None, 110.0)
(8, 2, 1, 2, 4.0)
(9, 1, 2, 3, 3.5)
(10, 2, 3, 1, 5.0)


In [11]:
# DEBUGGGGGGGGG

db('''
SELECT 
    id_plantacao, nome, id_lote, area
FROM
    plantacao
    INNER JOIN lote USING (id_plantacao)
ORDER BY
    id_lote
''')

Executando query:
(2, 'soja', 1, 100.0)
(1, 'milho', 2, 70.0)
(4, 'algodao', 4, 90.0)
(3, 'cafe', 5, 50.0)
(2, 'soja', 6, 30.0)
(4, 'algodao', 7, 110.0)
(1, 'milho', 8, 4.0)
(2, 'soja', 9, 3.5)
(3, 'cafe', 10, 5.0)


In [12]:
# MINHA RESPOSTAAAAAAA:

db('''
SELECT 
    id_plantacao, nome, id_lote, SUM(area) as area_total
FROM
    plantacao
    INNER JOIN lote USING (id_plantacao)
GROUP BY 
    id_plantacao
HAVING 
    area_total > 10
''')




Executando query:
(1, 'milho', 2, 74.0)
(2, 'soja', 1, 133.5)
(3, 'cafe', 5, 55.0)
(4, 'algodao', 4, 200.0)


In [13]:
# RESPOSTA DO PROFFFFFFF:

db('''
SELECT
    plantacao.nome, SUM(area) AS total
FROM
    lote,
    plantacao
WHERE
    lote.id_plantacao = plantacao.id_plantacao
GROUP BY
    lote.id_plantacao
HAVING total > 10;
''')

Executando query:
('milho', 74.0)
('soja', 133.5)
('cafe', 55.0)
('algodao', 200.0)


c) Qual o custo total atual com inseticidas?

In [14]:
#DEBUGGGG
#OBS não basta que o id do inseticida do lote seja o mesma da indicacao para saber a frequencia
# A plantacao também tem que bater, se não ele vai relacionar com todas as plantacoes com aquele id_inseticida
db('''
SELECT
     id_lote, nome, preco, area, frequencia
FROM
    plantacao_inseticida
    INNER JOIN inseticida USING (id_inseticida)
    INNER JOIN lote USING (id_inseticida)
WHERE
    lote.id_plantacao = plantacao_inseticida.id_plantacao
ORDER BY
    id_lote
''')

Executando query:
(1, 'Carinho', Decimal('700.00'), 100.0, 1)
(2, 'Mae-Terra', Decimal('500.00'), 70.0, 1)
(5, 'Carinho', Decimal('700.00'), 50.0, 1)
(6, 'Mae-Terra', Decimal('500.00'), 30.0, 1)
(8, 'Mae-Terra', Decimal('500.00'), 4.0, 1)
(9, 'Carinho', Decimal('700.00'), 3.5, 1)
(10, 'Mata-Tudo', Decimal('200.00'), 5.0, 4)


In [15]:
# MINHA RESPOSTAAAAAAAA (ESSE INNER JOIN DUPLO TÁ CERTO???):

db('''
SELECT
    SUM(preco*area*frequencia*12) as custo_total_inseticida
FROM
    plantacao_inseticida
    INNER JOIN inseticida USING (id_inseticida)
    INNER JOIN lote USING (id_inseticida, id_plantacao)
''')

Executando query:
(1961400.0,)


In [16]:
# MINHA RESPOSTA MAIS CONFIANTEEEEE
db('''
SELECT
    SUM(preco*area*frequencia*12) as custo_total_inseticida
FROM
    plantacao_inseticida
    INNER JOIN inseticida USING (id_inseticida)
    INNER JOIN lote USING (id_inseticida)
WHERE
    lote.id_plantacao = plantacao_inseticida.id_plantacao
''')


Executando query:
(1961400.0,)


In [17]:
# RESPOSTA DO PROFFFFFF:

db('''
SELECT
    SUM(lote.area *plantacao_inseticida.frequencia *inseticida.preco * 12)
FROM
    lote,
    inseticida,
    plantacao_inseticida
WHERE
    lote.id_inseticida = plantacao_inseticida.id_inseticida
    AND lote.id_plantacao = plantacao_inseticida.id_plantacao
    AND lote.id_inseticida = inseticida.id_inseticida;
''')


Executando query:
(1961400.0,)


In [18]:
# Primeiro vamos ver o menor custo de inseticida por plantacao levando em conta preço e frequencia
db('''
SELECT
   SUM(12*area*preco*frequencia) 
FROM
    lote
    INNER JOIN inseticida USING (id_inseticida)
    INNER JOIN plantacao_inseticida USING (id_inseticida, id_plantacao)

''')

Executando query:
(1961400.0,)


d) Quais inseticidas não foram usados?

In [19]:
#Criando dois útlimos que não tão sendo usados
db('''
INSERT INTO inseticida (id_inseticida, nome, preco) VALUES
(4, "AgroTop", 600),
(5, "Mata-Pouco", 100)
''')

Executando query:


In [20]:
db('''
SELECT
    *
FROM 
    inseticida
''')

Executando query:
(1, 'Mata-Tudo', Decimal('200.00'))
(2, 'Mae-Terra', Decimal('500.00'))
(3, 'Carinho', Decimal('700.00'))
(4, 'AgroTop', Decimal('600.00'))
(5, 'Mata-Pouco', Decimal('100.00'))


In [21]:
db('''
SELECT
    nome, id_lote
FROM
    inseticida
    LEFT OUTER JOIN lote USING (id_inseticida)

''')

Executando query:
('Mata-Tudo', 10)
('Mae-Terra', 2)
('Mae-Terra', 6)
('Mae-Terra', 8)
('Carinho', 1)
('Carinho', 5)
('Carinho', 9)
('AgroTop', None)
('Mata-Pouco', None)


In [22]:
db('''
SELECT
    nome, id_lote
FROM
    inseticida
    LEFT OUTER JOIN lote USING (id_inseticida)
WHERE 
    id_lote is NULL

''')

Executando query:
('AgroTop', None)
('Mata-Pouco', None)


e) Qual o menor custo possível com inseticidas? 

In [23]:
# Primeiro vamos ver o menor custo de inseticida por plantacao levando em conta preço e frequencia
db('''
SELECT
    id_plantacao, nome, frequencia, preco, MIN(preco*frequencia) as custo_preco_frequencia
FROM
    inseticida
    INNER JOIN plantacao_inseticida USING (id_inseticida)
GROUP BY 
    id_plantacao
''')

Executando query:
(1, 'Mata-Tudo', 2, Decimal('200.00'), Decimal('400.00'))
(2, 'Mae-Terra', 1, Decimal('500.00'), Decimal('500.00'))
(3, 'Mata-Tudo', 4, Decimal('200.00'), Decimal('700.00'))


In [24]:
db('DROP TEMPORARY TABLE IF EXISTS menor_preco_freq_por_plantacao')

db('''
CREATE TEMPORARY TABLE menor_preco_freq_por_plantacao
    SELECT
        id_plantacao, nome, frequencia, preco, MIN(preco*frequencia) as custo_preco_frequencia
    FROM
        inseticida
        INNER JOIN plantacao_inseticida USING (id_inseticida)
    GROUP BY 
        id_plantacao
''')

Executando query:
Executando query:


In [25]:
#Vendo se criou a tabela

db('SELECT * FROM menor_preco_freq_por_plantacao ')


Executando query:
(1, 'Mata-Tudo', 2, Decimal('200.00'), Decimal('400.00'))
(2, 'Mae-Terra', 1, Decimal('500.00'), Decimal('500.00'))
(3, 'Mata-Tudo', 4, Decimal('200.00'), Decimal('700.00'))


In [26]:
# Agora queremos achar todos os lotes, conferir as plantacoes, e ai usar o custo_preco_frequencia para calcular

db('''
SELECT
    id_lote, menor_preco_freq_por_plantacao.nome, custo_preco_frequencia, SUM(custo_preco_frequencia*area*12) as total_custo_lote 
FROM 
   menor_preco_freq_por_plantacao,
   lote
WHERE
    lote.id_plantacao = menor_preco_freq_por_plantacao.id_plantacao
    AND lote.id_inseticida IS NOT NULL

''')

Executando query:
(2, 'Mata-Tudo', Decimal('400.00'), 1618200.0)


In [27]:
connection.close()